### Imports

In [1]:
# Import the required modules
import cv2
from IPython.display import clear_output
import time
import PIL.Image
from io import BytesIO
import IPython.display
import numpy as np
import glob

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

----

### Open File Choose screen

In [2]:
%gui qt

from PyQt5.QtWidgets import QFileDialog

def gui_fname(dir=None):
    """Select a file via a dialog and return the file name."""
    if dir is None: dir ='./'
    fname = QFileDialog.getOpenFileName(None, "Select data file...", 
                dir, filter="All files (*);; SM Files (*.sm)")
    return fname[0]

### Open WebCAM from OpenCV on Jupyter Notebook

In [3]:
#Use 'jpeg' instead of 'png' (~5 times faster)
def array_to_image(a, fmt='jpeg'):
    #Create binary stream object
    f = BytesIO()
    
    #Convert array to binary stream object
    PIL.Image.fromarray(a).save(f, fmt)
    
    return IPython.display.Image(data=f.getvalue())

In [4]:
def get_frame(cam):
    # Capture frame-by-frame
    ret, frame = cam.read()
    
    #flip image for natural viewing
    frame = cv2.flip(frame, 1)
    
    return frame

In [5]:
def show_webcam():
    cam = cv2.VideoCapture(0)

    d = IPython.display.display("", display_id=1)
    d2 = IPython.display.display("", display_id=2)
    predict_data_steps = 0
    
    while True:
        try:
            t1 = time.time()
            frame = get_frame(cam)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            predict_data_steps += 1
            
            #if predict_data_steps % 10 == 0:
            frame = detect_face_cnn(frame)
                
            im = array_to_image(frame)

            d.update(im)

            t2 = time.time()

            s = f"""{int(1/(t2-t1))} FPS"""
            d2.update( IPython.display.HTML(s) )
        except KeyboardInterrupt:
            print()
            cam.release()
            IPython.display.clear_output()
            print ("Stream stopped")
            break

----

### Detect faces on image using OpenCV

In [6]:
# Face detection with OpenCV and deep learning (Adrian)
# https://www.pyimagesearch.com/2018/02/26/face-detection-with-opencv-and-deep-learning/

In [7]:
# load our serialized model from disk
caffe_model = 'deploy.prototxt.txt'
caffe_trained = 'res10_300x300_ssd_iter_140000.caffemodel'
caffe_confidence = 0.30
model_folder = './model/'
mask_model = "mask_mobile_net.h5"

In [8]:
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(model_folder + caffe_model, 
                               model_folder + caffe_trained
                              )


model = load_model(model_folder + mask_model)

[INFO] loading model...


Detect faces on image and call mask predictor

In [9]:
def detect_face_cnn(image, save = False, show = False):
    
    if image is not None:
        (h, w) = image.shape[:2]
        
        image_resized = cv2.resize(image, (300, 300))

        blob = cv2.dnn.blobFromImage(image_resized, 
                                     1.0,
                                     (300, 300), 
                                     (104.0, 
                                      177.0, 
                                      123.0))
        net.setInput(blob)
        detections = net.forward()

        for i in range(0, detections.shape[2]):
            # extract the confidence (i.e., probability) associated with the prediction
            confidence = detections[0, 0, i, 2]
           
            # filter out weak detections by ensuring the `confidence` is
            # greater than the minimum confidence
            if confidence > caffe_confidence:
                # compute the (x, y)-coordinates of the bounding box for the
                # object
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                try:
                    img_crop =   image[startY-10:endY+10, startX-10:endX+10]

                    # predict mask or not
                    pred, pred_res = predict_mask(img_crop)
                    
                    log_text = "Face Detection confidence:{:2}".format(round(confidence,2))
                    log_text = log_text + ", mask detection:{:2}".format(pred_res)
                    print(log_text)

                    label = "MASK" if pred_res == 0 else "NO-MASK"
                    color = (0,0,255) if pred_res == 0 else (255,0,0)

                    cv2.putText(image, label, (startX, startY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    cv2.rectangle(image, (startX, startY), (endX, endY), color)
                except:
                    print("found crop errors {}".format(round(confidence,2)))

        #if save:
        #    end_file = file_name.split("\\")[-1]
        #    save_file_path = file_name + "\\{:05}_{}".format(end_file)
        #    cv2.imwrite(save_file_path,image_all)

        if show:
            cv2.imshow("Image", image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            
        return image
    else:
        print("image not found!")

Predict if face is using mask or not

In [10]:
def predict_mask(image):
    image = cv2.resize(image, (224, 224))
    image = image.astype("float") / 255.0
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    
    # make predictions on the input image
    pred = model.predict(image)
    pred_res = pred.argmax(axis=1)[0]
    
    return pred, pred_res

### Check image source from file or Webcam

In [11]:
# LOCAL or WEBCAM
SOURCE = "WEBCAM"

In [12]:
# select image or webcam
if SOURCE == 'LOCAL':
    file_name = gui_fname()
    image = cv2.imread(file_name)
    detect_face_cnn(image, show = True)
else:
    show_webcam()

Stream stopped


Check all files on test folder

In [14]:
files_path = './data/test/*'

files_list = glob.glob(files_path)

for i, file in enumerate(files_list):
    try:
        print(file)
        image = cv2.imread(file)
        detect_face_cnn(image, show = True)
    except Exception as ex:
        print(ex)    

./data/test\mask1.jpg
Face Detection confidence:0.9100000262260437, mask detection: 0
found crop errors 0.9100000262260437
./data/test\mask10.jpeg
Face Detection confidence:0.9599999785423279, mask detection: 0
Face Detection confidence:0.7200000286102295, mask detection: 0
./data/test\mask11.jpg
./data/test\mask12.jpg
Face Detection confidence:0.949999988079071, mask detection: 0
Face Detection confidence:0.6100000143051147, mask detection: 0
Face Detection confidence:0.47999998927116394, mask detection: 0
Face Detection confidence:0.41999998688697815, mask detection: 0
Face Detection confidence:0.36000001430511475, mask detection: 0
./data/test\mask13.jpg
Face Detection confidence:0.9900000095367432, mask detection: 0
./data/test\mask14.jpg
Face Detection confidence:0.9900000095367432, mask detection: 0
./data/test\mask15.jpg
Face Detection confidence:1.0, mask detection: 0
./data/test\mask16.jpg
Face Detection confidence:0.8799999952316284, mask detection: 0
./data/test\mask17.jpg
F